In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

In [0]:
import string

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
%cd /content/gdrive/My \Drive/PatternProject
!ls

/content/gdrive/My Drive/PatternProject
data.csv


Here we read the data and save it to the X and Y variables!



In [0]:
data = pd.read_csv('data.csv', keep_default_na=False)
X = data['Raw']
Y = data['Encrypt']

# **Character Based Methods - Simple Neural Network**
In this part we focused on the charachters being change in the encryption. There are lots of models that can be used for this problem like: single perceptron, fully connected neural networks and so on!

In [0]:
# we get all the upper case letters
characters = list(string.ascii_uppercase) + list(string.punctuation) + list(string.digits) +list(string.ascii_lowercase) + list(' ')
N = len(characters)
# print(characters)
# print(characters.index(str(1)))

In [0]:
def one_hot_vector(char):
  idx = characters.index(str(char)) 
  vector = np.zeros(N)
  vector[idx] = 1
  return vector

In [0]:
def one_hot_to_char(vect):
  result = np.where(vect == np.amax(vect))
  idx = result[1][0]
  return characters[idx]

In [0]:
def change_data(x, y):
  new_x, new_y = [], []
  for k in x:
    for e in k:
      vect = one_hot_vector(e)
      new_x.append(vect)
  new_x = np.array(new_x)

  for k in y:
      for e in str(k):
        vect = one_hot_vector(e)
        new_y.append(vect)
  new_y = np.array(new_y)

  return new_x, new_y

In [0]:
epochs = 10
batch_size = 256

In [0]:
def model():
  model = keras.Sequential([
        keras.layers.Input(shape=(N,)),
        keras.layers.Dense(32, activation='relu'),
        keras.layers.Dense(N, activation='sigmoid')
  ])
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  print(model.summary())
  return model

In [50]:
# creating the model
mymodel = model()
# preprocessing data accourding to model architecture
new_x, new_y = change_data(X, Y)
m = len(new_x)
split = int(0.8*m)
x_train, y_train, x_test, y_test = new_x[:split], new_y[:split], new_x[split:], new_y[split:]

mymodel.fit(x_train, y_train, epochs=epochs, batch_size=batch_size)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 32)                3072      
_________________________________________________________________
dense_3 (Dense)              (None, 95)                3135      
Total params: 6,207
Trainable params: 6,207
Non-trainable params: 0
_________________________________________________________________
None
Train on 987147 samples
Epoch 1/10
987147/987147 [==============================] - 14s 15us/sample - loss: 0.2781 - acc: 0.9828
Epoch 2/10
987147/987147 [==============================] - 14s 14us/sample - loss: 3.6279e-04 - acc: 1.0000
Epoch 3/10
987147/987147 [==============================] - 14s 14us/sample - loss: 7.3269e-05 - acc: 1.0000
Epoch 4/10
987147/987147 [==============================] - 14s 14us/sample - loss: 2.8815e-05 - acc: 1.0000
Epoch 5/10
987147/987147 [===========================

In [0]:
import pickle
pickle.dumps(mymodel)

Predicting a single charachter

In [82]:
# test the model here
test1 = one_hot_vector('X')
test1 = test1.reshape(1,N)
vect = mymodel.predict(test1)
print(one_hot_to_char(vect))

mystring = 'AABD./ N'
mystring = 'AAB./'
ans = ''
for c in mystring:
  test1 = one_hot_vector(c)
  test1 = test1.reshape(1,N)
  vect = mymodel.predict(test1)
  ans += one_hot_to_char(vect)
print(ans)

J
MMN./


Predicting test data and caculating the accuracy

In [0]:
pred_y = mymodel.predict(x_test)
for i in range(len(pred_y)):
  idx = np.argmax(pred_y[i])
  pred_y[i, idx] = 1

In [17]:
from sklearn.metrics import average_precision_score, accuracy_score, confusion_matrix

m = len(y_test)
pred_labels = [np.argmax(pred_y[i]) for i in range(m)]
test_labels = [np.argmax(y_test[i]) for i in range(m)]

# average_precision = average_precision_score(test_labels, pred_labels)
accuracy = accuracy_score(test_labels, pred_labels)
# cm = confusion_matrix(test_labels, pred_labels)

# print('The Average Score Of Precision and Recall is: ', str(average_precision))
print('The Score Of Accuracy is: ', str(accuracy))

The Score Of Accuracy is:  1.0


# **Using Deep Learning Models - CNN and Seq-to-Seq**
In this section we try to use different deep learning models such as CNN and sequence-to-sequence models. The intuition for using a seq2seq model is that we are trying to decode an encoded sequence of charachters. So it is important to capture the relations (if there are any!) between characters in a single encoded text. 

CNN are mostly used in image processing but they have been applied for text as well in character level. As we are focusing on characters in this project we can also use CNNs to see how it works. 

reading the data

In [0]:
max_len_tokens = 35

In [19]:
from keras.preprocessing.text import Tokenizer

# here we use keras library to get the one-hot vector of out input data. first we need to make our train and test data integers
# then we can use scikit learn to_categorical() to get the one-hot vector.
tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts(pd.concat([X, Y]))
number_of_chars = len(tokenizer.word_index)

m = len(X)
split = int(0.8*m)

# getting the sequence of numbers that represent the character
x_train = np.asarray(tokenizer.texts_to_sequences(X.iloc[:split]))
y_train = np.asarray(tokenizer.texts_to_sequences(Y.iloc[:split]))
x_test = np.asarray(tokenizer.texts_to_sequences(X.iloc[split:]))
y_test = np.asarray(tokenizer.texts_to_sequences(Y.iloc[split:]))

Using TensorFlow backend.


In [0]:
def string_to_int_sequence(s):
  arr = np.asarray(tokenizer.texts_to_sequences(s))
  return arr

In [0]:
# make the one-hot with padding
def make_arr(mat):
  u = len(mat)
  tmp = np.zeros((u, max_len_tokens, number_of_chars))
  for i in range(u):
    for j in range(len(mat[i])):
      tmp[i][j][mat[i][j]-1] = 1
  return tmp

In [0]:
# this is the list of all sequence of characters that are converted to one-hot and have padding
new_x_train = make_arr(x_train)
new_y_train = make_arr(y_train)
new_x_test = make_arr(x_test)
new_y_test = make_arr(y_test)

In [22]:
print(new_x_train.shape)

(107613, 35, 35)


**Sequence-to-Sequence Model**

In [0]:
def seq2seq(max_len_tokens):
  encoder_inputs = keras.layers.Input(shape=(max_len_tokens, number_of_chars), name='encoder_input')
  encoder = keras.layers.LSTM(300, return_state=True, name='encoder_lstm', activation='linear')
  encoder_outputs, state_h, state_c = encoder(encoder_inputs)
  
  encoder_states = [state_h, state_c]

  decoder_inputs = keras.layers.Input(shape=(max_len_tokens-1, number_of_chars), name='decoder_input')
  decoder_lstm = keras.layers.LSTM(300, return_sequences=True, return_state=True, name='decoder_lstm', activation='linear')
  decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
  decoder_dense = keras.layers.Dense(number_of_chars, activation='sigmoid', name='dense_layer')
  decoder_outputs = decoder_dense(decoder_outputs)

  model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  print(model.summary())

  return model

In [0]:
model = seq2seq(max_len_tokens)
decoder_input = new_y_train[:,0:max_len_tokens-1,:]
decoder_target = new_y_train[:,1:max_len_tokens,:]

callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

model.fit([new_x_train, decoder_input], decoder_target, batch_size=batch_size, epochs=1, callbacks=[callback], validation_split=0.2)

Model: "model_17"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      [(None, 35, 35)]     0                                            
__________________________________________________________________________________________________
decoder_input (InputLayer)      [(None, 34, 35)]     0                                            
__________________________________________________________________________________________________
encoder_lstm (LSTM)             [(None, 300), (None, 403200      encoder_input[0][0]              
__________________________________________________________________________________________________
decoder_lstm (LSTM)             [(None, 34, 300), (N 403200      decoder_input[0][0]              
                                                                 encoder_lstm[0][1]        

**Using CNN**

In [0]:
def cnn_model():
  model = keras.Sequential([
            keras.layers.Input(shape=(max_len_tokens, number_of_chars), name='input_layer'),
            keras.layers.Conv1D(strides=1, padding='same', activation='relu', kernel_size=3, filters=300, name='convolution_layer1'),
            keras.layers.MaxPool1D(),
            keras.layers.Conv1D(strides=1, padding='same', activation='relu', kernel_size=4, filters=300, name='convolution_layer2'),
            keras.layers.Dropout(0.5),
            keras.layers.Flatten(),
            keras.layers.Dense(number_of_chars, activation='relu', name='dense_layer')
  ])

  model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
  model.summary()

  return model 

In [0]:
# the target value is the vector of all integers that represent the int value of character
def make_y_vector(mat):
  l = len(mat)
  v = np.zeros((l, max_len_tokens))
  v = np.zeros((l, 1))
  for i in range(l):
    # idx = np.argmax(pred_y[i])
    # v[i][0] = idx 
    for j in range(len(mat[i])):
      v[i][j] = mat[i][j]
  return v

In [0]:
print(make_y_vector(y_train).shape)
print(y_train.shape)

In [27]:
cnnmodel = cnn_model()
callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
cnnmodel.fit(new_x_train, make_y_vector(y_train), epochs=epochs, batch_size=batch_size, callbacks=[callback], validation_split=0.2)
pickel.dumps(cnnmodel)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
convolution_layer1 (Conv1D)  (None, 35, 300)           31800     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 17, 300)           0         
_________________________________________________________________
convolution_layer2 (Conv1D)  (None, 17, 300)           360300    
_________________________________________________________________
dropout (Dropout)            (None, 17, 300)           0         
_________________________________________________________________
flatten (Flatten)            (None, 5100)              0         
_________________________________________________________________
dense_layer (Dense)          (None, 35)                178535    
Total params: 570,635
Trainable params: 570,635
Non-trainable params: 0
________________________________________________

In [35]:
from sklearn.metrics import average_precision_score, accuracy_score, confusion_matrix
# now it's time for testing
y_pred = cnnmodel.predict(new_x_test)
# # make the float value round (as they are all indexes)
for i in range(len(y_pred)):
  y_pred[i] = np.round(y_pred[i])

arr_y_test = make_y_vector(y_test).astype(int)
y_pred = y_pred.astype(int)

correct = 0
total = 0
for i in range(y_pred.shape[0]):
  for j in range(y_pred.shape[1]):
    if y_pred[i][j] == arr_y_test[i][j]:
      correct += 1
    total += 1

print('The Accuracy Of The Model Is: ', str(correct/total))

The Accuracy Of The Model Is:  0.8643558047661527


In [81]:
reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))
mystring = ['AAB./']
print(string_to_int_sequence(mystring))
s = make_arr(string_to_int_sequence(mystring))
pred = cnnmodel.predict(s)
ans = ''
for i in range(len(pred)):
  for x in pred[i]:
    if x > 0:
      ans += reverse_word_map[int(round(x))]

print(ans)

[[ 1  1 19 29 30]]
mmt'w
